In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import random
from shutil import copyfile

# Face Mask Detector Using Tensorflow and OpenCV

### Number of With mask vs Without Mask

In [2]:
print("With Mask: ",len(os.listdir('dataset/with_mask')))
print("Without Mask: ",len(os.listdir('dataset/without_mask')))

With Mask:  1916
Without Mask:  1919


### Creating Train and Test Data

In [3]:
def split_data(SOURCE, TRAIN, TEST, SPLIT_SIZE):
    proper_data = []
    for img in os.listdir(SOURCE):
        img_path = SOURCE + img
        if(os.path.getsize(img_path)>0):
            proper_data.append(img)
        else:
            print("Issue in the Image:"+ img_path)
            
    training_size = int(len(proper_data)*SPLIT_SIZE)
    shuffled_data = random.sample(proper_data, len(proper_data))
    training_data = shuffled_data[:training_size]
    testing_data = shuffled_data[training_size:]
    #print(training_data)
    for img in training_data:
        temp_img = SOURCE + img
        train_img = TRAIN + img
        copyfile(temp_img, train_img)

    for img in testing_data:
        temp_img = SOURCE + img
        test_img = TEST + img
        copyfile(temp_img, test_img)

In [4]:
YES_Source = 'dataset/with_mask/'
TRAINING_YES_Source = 'dataset/train/with_mask/'
TESTING_YES_Source = 'dataset/test/with_mask/'
NO_Source = 'dataset/without_mask/'
TRAINING_NO_Source = 'dataset/train/without_mask/'
TESTING_NO_Source = 'dataset/test/without_mask/'
split_size = 0.8
split_data(YES_Source, TRAINING_YES_Source, TESTING_YES_Source, split_size)
split_data(NO_Source, TRAINING_NO_Source, TESTING_NO_Source, split_size)

In [5]:
print('Number of training with mask: ', len(os.listdir(TRAINING_YES_Source)))
print('Number of training without mask: ', len(os.listdir(TRAINING_NO_Source)))
print('Number of testing with mask: ', len(os.listdir(TESTING_YES_Source)))
print('Number of testing without mask: ', len(os.listdir(TESTING_NO_Source)))

Number of training with mask:  1916
Number of training without mask:  1914
Number of testing with mask:  1040
Number of testing without mask:  1025


### Creating CNN Model using TensorFlow

In [6]:
#Creating the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
TRAIN_DIR = 'dataset/train/'
train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    target_size = (150,150),
                                                    class_mode='categorical',
                                                    batch_size = 32)
TEST_DIR = 'dataset/test/'
test_datagen = ImageDataGenerator(rescale = 1/255.0)
test_generator = test_datagen.flow_from_directory(TEST_DIR,
                                                target_size = (150,150),
                                                class_mode='categorical',
                                                batch_size = 32)

Found 3828 images belonging to 2 classes.
Found 2065 images belonging to 2 classes.


In [9]:
#Used to get best weights used when monitored with validation loss
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [10]:
history = model.fit_generator(train_generator,
                             epochs = 30,
                             validation_data=test_generator,
                             callbacks=[checkpoint])

C:\Users\phatk\AppData\Local\Temp\ipykernel_29412\288473390.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


Epoch 1/30
 34/120 [=======>......................] - ETA: 16s - loss: 0.6417 - accuracy: 0.6737

C:\Users\phatk\anaconda3\envs\tf_gpu\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


120/120 [==============================] - ETA: 0s - loss: 0.4384 - accuracy: 0.8051

INFO:tensorflow:Assets written to: model-001.model\assets


...

INFO:tensorflow:Assets written to: model-027.model\assets


INFO:tensorflow:Assets written to: model-027.model\assets


120/120 [==============================] - 28s 234ms/step - loss: 0.1526 - accuracy: 0.9454 - val_loss: 0.0739 - val_accuracy: 0.9748
Epoch 28/30
 34/120 [=======>......................] - ETA: 15s - loss: 0.1650 - accuracy: 0.9393

C:\Users\phatk\anaconda3\envs\tf_gpu\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


120/120 [==============================] - 30s 246ms/step - loss: 0.1543 - accuracy: 0.9436 - val_loss: 0.0914 - val_accuracy: 0.9695
Epoch 29/30
120/120 [==============================] - 33s 272ms/step - loss: 0.1417 - accuracy: 0.9428 - val_loss: 0.0787 - val_accuracy: 0.9743
Epoch 30/30
120/120 [==============================] - 31s 261ms/step - loss: 0.1539 - accuracy: 0.9423 - val_loss: 0.1033 - val_accuracy: 0.9632


In [ ]:
'''Remove the commented Code lines to save your Video'''

labels = {0:'No Mask, Please wear it!',1:'Mask On, Good work!'}
color = {0:(0,0,255), 1:(0,255,0)}

webcam = cv2.VideoCapture(0)

classifier = cv2.CascadeClassifier('dataset/haarcascade_frontalface_default.xml')
fourcc = cv2.VideoWriter_fourcc(*'ADV1') ######
out = cv2.VideoWriter('lol.mp4', fourcc, 25, (640,480)) ######
while webcam.isOpened():
    _, frame = webcam.read()
    frame = cv2.flip(frame, 1, 1)
    faces = classifier.detectMultiScale(frame, 1.1, 5)
    
    for (x,y,w,h) in faces:
#         cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2) ######
        face_data = frame[y:y+h, x:x+w]
        resize_data = cv2.resize(face_data, (150,150))
        resize_data = resize_data/255.0
        final_data = np.expand_dims(resize_data, axis = 0)
        prediction = model.predict(final_data)
        
        binary_answer = np.argmax(prediction,axis=1)[0]
        
        cv2.rectangle(frame, (x,y), (x+w, y+h), color[binary_answer], 2)
#         cv2.rectangle(frame, (x,y-40), (x+w, y), color[binary_answer], -1) no solid
        cv2.putText(frame, labels[binary_answer], (x,y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)   

    cv2.imshow("Face mask detection", frame)
    out.write(frame) ######
    if cv2.waitKey(1) == 27:
            break  

out.release() ######
webcam.release()
cv2.destroyAllWindows()